# This notebook contains the experiments on Heart Statlog dataset with LionForests

In [1]:
from LionForests import LionForests
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
import urllib

Firstly, we load the dataset and we set the feature and class names

In [2]:
url="http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat"
raw_data = urllib.request.urlopen(url)
credit=np.genfromtxt(raw_data)
X,y = credit[:,:-1], credit[:,-1].squeeze()
feature_names = ['age','sex','chest pain','resting blood pressure','serum cholestoral',
               'fasting blood sugar','resting electrocardiographic results','maximum heart rate achieved','exercise induced angina','oldpeak',
               'the slope of the peak exercise','number of major vessels','reversable defect']
class_names = ['absence','presence']

This dataset contains few instances. Only 270

In [3]:
len(X)

270

We can explore the features of this dataset

In [4]:
pd.DataFrame(X,columns=feature_names).describe()

,age,sex,chest pain,resting blood pressure,serum cholestoral,fasting blood sugar,resting electrocardiographic results,maximum heart rate achieved,exercise induced angina,oldpeak,the slope of the peak exercise,number of major vessels,reversable defect
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,3.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,1.585185,0.670370,4.696296
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,1.940659
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,1.000000,0.000000,3.000000
25%,48.000000,0.000000,3.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,1.000000,0.000000,3.000000
50%,55.000000,1.000000,3.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,2.000000,0.000000,3.000000
75%,61.000000,1.000000,4.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,2.000000,1.000000,7.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,3.000000,3.000000,7.000000


Then, we can train our random forests model using LionForests

In [5]:
y = [int(i-1) for i in y] 
parameters = [{
    'max_depth': [5],
    'max_features': ['sqrt'],
    'bootstrap': [False],
    'min_samples_leaf' : [5],
    'n_estimators': [500]
}]
lf = LionForests(class_names=class_names)
scaler = MinMaxScaler(feature_range=(-1,1))
lf.train(X, y, scaler, feature_names, parameters)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.8s finished


And we can see the number of estimators and the best set of parameters

In [6]:
number_of_estimators = lf.model.n_estimators
print("Accuracy:",lf.accuracy,", Number of estimators:",lf.number_of_estimators)
print(lf.model)

Accuracy: 0.8188916011524707 , Number of estimators: 500
RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


Now, we are ready to produce explanations using lionForests

In [7]:
rule = lf.following_breadcrumbs(X[25], False, True, False, complexity=4)
print(rule)

['if 3.0<=reversable defect<=4.5 & 2.5<=chest pain<=3.49 & 0.0<=number of major vessels<=0.51 & 0.124<=oldpeak<=0.248 & 0.0<=exercise induced angina<=0.5 & 137.81<=maximum heart rate achieved<=140.43 & 1.0<=the slope of the peak exercise<=1.5 & 47.72<=age<=49.64 & 272.73<=serum cholestoral<=279.3 & 0.0<=sex<=0.5 then absence', 490, 13, 251, 10]


In [8]:
discrete_features = ['age']
lf.check_changes_in_prediction(X[25], rule, discrete_features)

---------FEATURES THAT GOT REDUCED FROM LF BELOW---------
resting blood pressure
fasting blood sugar
resting electrocardiographic results
---------NEW FEATURE VALUES BELOW [left,middle,right]---------
age [46, 48, 50]
sex ['0.0000', '0.2516', '0.7516']
chest pain ['1.6266', '3.4367', '3.9267']
serum cholestoral ['149.6525', '279.2441', '525.4550']
maximum heart rate achieved ['89.1882', '140.4162', '189.0518']
exercise induced angina ['0.0000', '0.2630', '0.7630']
oldpeak ['0.0042', '0.2479', '1.9256']
the slope of the peak exercise ['1.0000', '1.4540', '2.8621']
number of major vessels ['0.0000', '0.1969', '0.7069']
reversable defect ['3.0000', '4.3946', '6.8243']
---------FEATURE VALUES THAT MAY CHANGE THE CLASSIFICATION BELOW---------


{0: [46, 48, 50],
 1: ['0.0000', '0.2516', '0.7516'],
 2: ['1.6266', '3.4367', '3.9267'],
 4: ['149.6525', '279.2441', '525.4550'],
 7: ['89.1882', '140.4162', '189.0518'],
 8: ['0.0000', '0.2630', '0.7630'],
 9: ['0.0042', '0.2479', '1.9256'],
 10: ['1.0000', '1.4540', '2.8621'],
 11: ['0.0000', '0.1969', '0.7069'],
 12: ['3.0000', '4.3946', '6.8243']}

In [9]:
#tests below

#X[25][2] = 1.5696
#lf.following_breadcrumbs(X[25], False, True, False, complexity=4)

#X[25][0] = 46
#lf.following_breadcrumbs[X[25], False, True, False, complexity=4]

#X[25][9] = 0.0197
#lf.following_breadcrumbs[X[25], False, True, False, complexity=4]

#X[25][12] = 6.7361
#lf.following_breadcrumbs[X[25], False, True, False, complexity=4]
